In this notebook, I show the result of training/using the neural network only on the lowest 28 grid points.

In [ ]:
%matplotlib inline

In [ ]:
import xarray as xr
import holoviews as hv
hv.extension('bokeh')
from gnl.colorblind import colorblind
colorblind()

# Short time simulation

In [ ]:
id = "kickass_baekeland"
# id = "elated_leavitt"
def open_id(id):
    ds3d = xr.open_mfdataset(f"../data/samNN/{id}/OUT_3D/*.nc")
    ds2d = xr.open_mfdataset(f"../data/samNN/{id}/OUT_2D/*.nc")

    return xr.merge([ds3d, ds2d])

In [ ]:
ds = open_id(id)

In [ ]:
ds.W[::3,5].plot(col='time', col_wrap=5)

In [ ]:
ds.SLI[::3,5].plot(col='time', col_wrap=5)

In [ ]:
ds.Prec[::6].plot(col='time', col_wrap=5)

In [ ]:
ds.Prec[::5].mean(['x']).plot(x='y', hue='time');

#  Longer simulation

I tried to run a 100 day simulation, but it died after only 6 days

In [ ]:
d2d = xr.open_mfdataset("/Users/noah/Data/0/7a/194be4f48e151e4391a229780b0e69/OUT_2D/*.nc")

In [ ]:
d2d.time.max()

This shows the simulation only lasted about 8 days. I am not sure why it stopped running.

In [ ]:
d2d.Prec[::24].plot(col='time', col_wrap=3)

In [ ]:
d2d.PW[::24].plot(col='time', col_wrap=3)

Eventually the precipitable water field becomes very grainy.

In [ ]:
d2d.LHF[::24].plot(col='time', col_wrap=3)

the other fields are also super grainy.

In [ ]:
d2d.V200[:72:12].plot(col='time', col_wrap=3)

In [ ]:
d2d.U200[::24].plot(col='time', col_wrap=3)

It seems like the model is not dissipative enough. I should run it withhout the neural network to see what happens. The graininess is very clear after 3 days, so that should be long enough.

In [ ]:
d2d.U850[::24].plot(col='time', col_wrap=3)

# Initial value, no physics

```
dopython = .false.
dosgs = .false.
dodamping = .false.
```

In [ ]:
ds = xr.open_mfdataset("../data/samNN/mad_noyce/OUT_2D/*.nc")

In [ ]:
ds.PW[::24].plot(col='time', col_wrap=3)

In [ ]:
ds.V200[::12].plot(col='time', col_wrap=3)

Without the neural network, the fields are not as grainy, but there still seems to be too little dissipation. I will try running the scheme with the active NN and upper atmospheric damping.

# NN on with sponge layer

In [ ]:
ds = xr.open_mfdataset("../data/samNN/prickly_brattain/OUT_2D/*.nc")

Here is the precipitation field:

In [ ]:
ds.Prec[::12].plot(col='time', col_wrap=3)

In [ ]:
ds.V200[::12].plot(col='time', col_wrap=3)

Compared to the older simulation with damping off:

In [ ]:
d2d.V200[:72:12].plot(col='time', col_wrap=3)

The simulation with the sponge layeris slightly less grainy. But the model is still not dissipative enough. Chris has repeatedly mentioned that we will need to parametrize the momentum source. Maybe that would help damp this model adequately. I don't think this is a grid-scale instability because the precipitation field is not diverging too quickly.